In [1]:
import requests
import visvalingamwyatt as vw
import json
import time
import subprocess
import os

DATA_DIRECTORY = os.path.join(os.getcwd(), "data")
MAPSHAPER = os.path.join(os.path.expanduser('~'), "node_modules/mapshaper/bin/mapshaper")
API_KEY = "ADD AN OPENROUTESERVICE API KEY HERE"

DIRECTIONS_API = 'https://api.openrouteservice.org/v2/directions/driving-car/geojson'
ISOCHRONES_API = 'https://api.openrouteservice.org/v2/isochrones/driving-car/geojson'

In [16]:
grocery_stores = [
    [[-80.9517330317152, 34.14295707100237]],
    [[-80.99471857647124, 34.09164323815291], [-80.96636935373824, 34.137998812686284]],
    [[-81.02692251578029, 34.0527402547101]],
    [[-81.0090162534106, 34.06332759023349]]
]

dollar_stores = [
    [[-81.042723, 34.04526]],
    [[-81.041781, 34.030223]],
    [[-81.024921, 34.029288]],
    [[-81.026992, 34.056078]],
    [[-81.008434, 34.061981]],
    [[-81.055619, 34.069414]],
    [[-80.985022, 34.1028]],
    [[-81.014754, 34.05705]],
    [[-80.995886, 34.086025]]
]

dialysis_centers = [
    # 3900 N Main St a, Columbia, SC 29203
    [[-81.0405388590042, 34.03762746796929]],
    # 7631 Wilson Blvd, Columbia, SC 29203
    [[-80.99752497202587, 34.08011407958197]],
    # 3511 Medical Dr, Columbia, SC 29203
    [[-81.03690200348677, 34.0315342047805]],
    # Two Medical Park, 2 Medical Park Rd, Columbia, SC 29203
    [[-81.0357269071416, 34.02727788870631]],
    # 140 Highland Center Drive, Columbia, SC 29203
    [[-80.95013973916826, 34.11443114414363]],
    # 7499 Parklane Rd, Columbia, SC 29223
    [[-80.95817645743686, 34.0785016723008]],
    # 2125 Adams Grove, Columbia, SC 29203
    [[-81.0259927004938, 34.027421134870536]],
    # 121 Park Central Dr, Columbia, SC 29203
    [[-81.0403318925324, 34.02934966876713]],
    # 2139 Adams Grove, Columbia, SC 29203
    [[-81.02512424041745, 34.027394522314935]]
]

### (1) Find Isochrones for Grocery Stores vs. Dollar Stores

In the Eau Claire community in Columbia, SC, we are interested in the proximity of residents to dollar stores vs proximity to grocery stores. Using the USDA's Low-Income, Low-Access definition, we are minimally interested in the distances at 1/2 mile and 1 mile. For low-income residents with low access to transportation, it's critical to be within 1/2 mile or at least 1 mile of a grocery store.

To help find these distances, we started with a dataset of buildings in the Columbia, SC area derived from [Microsoft's Building Footprint Data](https://github.com/Microsoft/USBuildingFootprints). We used the Mapshaper GUI to take the full dataset, clip it by a Columbia, SC shapefile, and output only the Columbia, SC buildings. The downside of the data is that it does not indicate which buildings are residential, so ultimately, the maps are more of a proxy than an exact representation of residential distance to stores.

Here, we're clipping the building data further to only include the greater Eau Claire area. After clipping, we then use an isochrone API from [Openrouteservice](https://openrouteservice.org/dev/#/api-docs/v2/isochrones) to create isochrone polygons (a polygon whose bounds are X miles away from a specified point) for all dollar stores and grocery stores. We then clip the building data by isochrones at 1/4 mile, 1/2 mile, 3/4 mile, and 1 mile distances away from the specific point. We output that data, reaggregate it and label each building by which isochrone zone it is in.

Ultimately each building will be tagged with a label indicating which isochrone it was grouped into. For instance, the isochrone tags for how far a building is from a grocery store would be the following. The value after the dash indicates the number of meters away from the grocery store since Openrouteservice uses meters. 
    
    grocery-403
    grocery-805
    grocery-1280
    grocery-1610

Metric units to Imperial units guide:

    403 meters ~ 1/4 mile
    805 meters ~ 1/2 mile
    1280 meters ~ 3/4 mile
    1610 meters ~ 1 mile
    
If the tag is empty, then the building is greater than 1610 meters (~1 mile) away from a grocery store.

In [17]:
eau_claire_filepath = os.path.join(DATA_DIRECTORY, 'eau_claire.json')
bldgs_columbia_filepath = os.path.join(DATA_DIRECTORY, 'buildings_columbia.json')
bldgs_eau_claire_filepath = os.path.join(DATA_DIRECTORY, 'buildings_eau_claire.json')

subprocess.run([MAPSHAPER, bldgs_columbia_filepath, "-clip", eau_claire_filepath, "-o", bldgs_eau_claire_filepath])

[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/buildings_eau_claire.json


CompletedProcess(args=['/Users/davidnewcomb/node_modules/mapshaper/bin/mapshaper', '/Users/davidnewcomb/notebooks/data/diabetic_amputations/buildings_columbia.json', '-clip', '/Users/davidnewcomb/notebooks/data/diabetic_amputations/eau_claire.json', '-o', '/Users/davidnewcomb/notebooks/data/diabetic_amputations/buildings_eau_claire.json'], returncode=0)

In [18]:
# ISOCHRONES - calculated from Openrouteservice
def output_isochrone_polygons(locations, distances, location_type):
    output_directory = os.path.join(DATA_DIRECTORY, 'isochrones')
    for i, location in enumerate(locations):
        for distance in distances:
            body = {"locations":[location],"range":[distance],"range_type": "distance"}

            headers = {
                'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
                'Authorization': API_KEY,
                'Content-Type': 'application/json; charset=utf-8'
            }
            call = requests.post(
                ISOCHRONES_API,
                json=body,
                headers=headers
            )
            data = json.loads(call.text)
            
            output_filepath = os.path.join(output_directory, 'isochrone_{0}_{1}_{2}m.json'.format(
                location_type,
                i,
                distance
            ))

            with open(output_filepath, 'w') as f:
                json.dump(data, f)
                
            print('Finished isochrone output for {0}_{1} at {2} meters'.format(location_type, i, distance))
            # Openrouteservice API has a limit of 20 requests per minute for Isochrones
            # (i.e. 1 request per 3 seconds)
            time.sleep(3)

# ~1/4 mile, 1/2 mile, ~3/4 mile, 1 mile
isochrone_distances = [403, 805, 1208, 1610]
isochrone_grocery_locations = [
    [-81.02692251578029, 34.0527402547101],
    [-81.0090162534106, 34.06332759023349]
]
isochrone_bargain_locations = [
    [-81.042723, 34.04526],
    [-81.041781, 34.030223],
    [-81.024921, 34.029288],
    [-81.026992, 34.056078],
    [-81.008434, 34.061981],
    [-81.055619, 34.069414],
    [-81.014754, 34.05705],
    [-81.009038, 34.063227],
    [-80.995886, 34.086025],
    [-81.00930364148168, 34.036282996659864],
    [-81.01493384807655, 34.02040201801057]
]
isochrone_config = {
    'grocery': {
        'distances': isochrone_distances,
        'locations': isochrone_grocery_locations,
    },
    'bargain': {
        'distances': isochrone_distances,
        'locations': isochrone_bargain_locations,
    }
}

for key, value in isochrone_config.items():
    distances = value['distances']
    locations = value['locations']
    location_type = key
    output_isochrone_polygons(locations, distances, location_type)


Finished isochrone output for grocery_0 at 403 meters
Finished isochrone output for grocery_0 at 805 meters
Finished isochrone output for grocery_0 at 1208 meters
Finished isochrone output for grocery_0 at 1610 meters
Finished isochrone output for grocery_1 at 403 meters
Finished isochrone output for grocery_1 at 805 meters
Finished isochrone output for grocery_1 at 1208 meters
Finished isochrone output for grocery_1 at 1610 meters
Finished isochrone output for bargain_0 at 403 meters
Finished isochrone output for bargain_0 at 805 meters
Finished isochrone output for bargain_0 at 1208 meters
Finished isochrone output for bargain_0 at 1610 meters
Finished isochrone output for bargain_1 at 403 meters
Finished isochrone output for bargain_1 at 805 meters
Finished isochrone output for bargain_1 at 1208 meters
Finished isochrone output for bargain_1 at 1610 meters
Finished isochrone output for bargain_2 at 403 meters
Finished isochrone output for bargain_2 at 805 meters
Finished isochrone o

In [19]:
# Get ids of buildings which are found in isochrone polygons
isochrone_ids = {}

for key, value in isochrone_config.items():
    distances = sorted(value['distances'])
    locations = value['locations']
    location_type = key

    output_directory = os.path.join(DATA_DIRECTORY, "isochrones")
    temp_filepath = os.path.join(output_directory, "temp.json")
    
    isochrone_ids[location_type] = {}
    for i, distance in enumerate(distances):
        isochrone_ids[location_type][distance] = []
        distance_ids = set()
        for j, location in enumerate(locations):
            isochrone_filepath = os.path.join(output_directory, 'isochrone_{0}_{1}_{2}m.json'.format(
                location_type,
                j,
                distance
            ))

            # Clip shapes by the given isochrone
            subprocess.run([
                MAPSHAPER,
                bldgs_eau_claire_filepath,
                "-clip",
                isochrone_filepath,
                "-o",
                temp_filepath
            ])

            # Read the temp file to gather the IDs contained by the isochrone
            with open(temp_filepath) as f:
                features = json.load(f)
                
            geometries = features['objects']['bldg_footprints']['geometries']
            ids = [geo['properties']['id'] for geo in geometries]
            distance_ids = distance_ids.union(set(ids))
        
        # Remove any ids that are closer to the point in question
        if (i > 0):
            prev_distance_ids = set()
            for prev_distance in distances[0:i]:
                prev_distance_ids = prev_distance_ids.union(isochrone_ids[location_type][prev_distance])
            distance_ids = distance_ids - prev_distance_ids
        
        isochrone_ids[location_type][distance] = distance_ids

[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/isochrones/temp.json
[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/isochrones/temp.json
[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/isochrones/temp.json
[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/isochrones/temp.json
[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/isochrones/temp.json
[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/isochrones/temp.json
[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/isochrones/temp.json
[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/isochrones/temp.json
[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/isochrones/temp.json
[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/isochrones/temp.json
[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/isochrones/temp.json
[o] Wrote /Users/davidnewcomb/notebooks/dat

In [20]:
with open(bldgs_eau_claire_filepath) as f:
    bldgs_eau_claire = json.load(f)

In [21]:
# This is where we're tagging buildings by which isochrone zone they fall in
# for both grocery stores and bargain stores.
for i, geo in enumerate(bldgs_eau_claire['objects']['bldg_footprints']['geometries']):
    geo_id = geo['properties']['id']
    for location_type in isochrone_config.keys():
        found_id = False
        for distance in isochrone_distances:
            if geo_id in isochrone_ids[location_type][distance]:
                geo['properties'][location_type] = distance
                found_id = True
                break
        if not found_id:
            bldgs_eau_claire['objects']['bldg_footprints']['geometries'][i]['properties'][location_type] = ''
        else:
            bldgs_eau_claire['objects']['bldg_footprints']['geometries'][i]['properties'][location_type] = '{0}-{1}'.format(location_type, distance)

In [22]:
with open(bldgs_eau_claire_filepath, 'w') as f:
    json.dump(bldgs_eau_claire, f)

### (2) Driving Routes to Grocery Stores, Dollar Stores & Dialysis Centers

Here we use the [Openrouteservice Directions API](https://openrouteservice.org/dev/#/api-docs/directions) to find driving directions from a single point to grocery stores, dollar stores and dialysis centers. I'm also simplifying the point vectors to make them less onerous for mapping. Need to be aware of rate limits - not implemented here.

In [15]:
def output_routes_from_point(point, destinations, filename):
    output_filepath = os.path.join(DATA_DIRECTORY, 'routes', 'routes_{0}.json'.format(filename))
    
    feature_collection = {
        "type": "FeatureCollection",
        "features": []
    }
    for destination in destinations:
        body = {"coordinates":(point + destination)}
        headers = {
            'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
            'Authorization': API_KEY,
            'Content-Type': 'application/json; charset=utf-8'
        }
        call = requests.post(
            DIRECTIONS_API,
            json=body,
            headers=headers
        )
        
        # Simplifying coordinates - may need to play around with the ratio for your purposes
        simplified_coordinates = vw.simplify_geometry(json.loads(call.text)['features'][0]['geometry'], ratio=.2)
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": simplified_coordinates['coordinates']
            }
        }
        feature_collection['features'].append(feature)

    with open(output_filepath, 'w') as f:
        json.dump(feature_collection, f)
    
    print("Finished outputting routes for {0}".format(filename))
        
# Representative home location in Eau Claire
home = [[-81.03660622747452, 34.037782522508856]]

route_locations = {
    "grocery": grocery_stores,
    "dollar_store": dollar_stores,
    "dialysis": dialysis_centers 
}

for key, locations in route_locations.items():
    output_routes_from_point(home, locations, key)

Finished outputting routes for grocery
Finished outputting routes for dollar_store
Finished outputting routes for dialysis


### (3) Transit Routes

This was an attempt to use the [HERE Maps API](https://developer.here.com/documentation/public-transit/dev_guide/routing/shape-example.html) for transit directions. It didn't have very useful coverage for the datapoints we were examining. A better bet would probably be to use Google Maps Transit if their stringent licensing ends up working for y'all.

In [ ]:
def stringify_array(array):
    return [str(item) for item in array]

import flexpolyline as fp

api_prefix = 'https://transit.router.hereapi.com/v8/routes'
api_key = 'SIGN UP AND GET AN API KEY'
origin = ','.join(stringify_array(reversed(home)))
dest = ','.join(stringify_array(reversed(dialysis_centers[1])))
transit_endpoint = "{0}?apiKey={1}&origin={2}&destination={3}&return=polyline".format(
    api_prefix,
    api_key,
    origin,
    dest
)

response = requests.get(transit_endpoint)
routes = json.loads(response.text)['routes']
primary_route = routes[0]
primary_sections = primary_route['sections']
for section in primary_sections:
    polyline = fp.decode(section['polyline'])
    print(polyline)